# Content Creation at Scale 📝

## Initial Imports

In [ ]:
%pip install "crewai[tools]"
%pip install "crewai[all]"

In [1]:
#Warning 
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    # DirectoryReadTool,
    # FileReadTool,
    SerperDevTool,
    ScrapeWebsiteTool, 
    WebsiteSearchTool
)

from helper import get_openai_api_key, get_serper_api_key
# Set up API keys
#os.environ["SERPER_API_KEY"] = "Your Key" # serper.dev API key
#os.environ["OPENAI_API_KEY"] = "Your Key"
os.environ["OPENAI_API_KEY"] = get_openai_api_key()
os.environ["SERPER_API_KEY"] = get_serper_api_key()

# Instantiate tools
#docs_tool = DirectoryReadTool(directory='./blog-posts')
#file_tool = FileReadTool()
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
web_rag_tool = WebsiteSearchTool()

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [ ]:
from helper import get_groq_api_key

In [ ]:
#os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
#os.environ["GROQ_API_KEY"] = get_groq_api_key()
#groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [ ]:
from helper import get_groq_api_key

In [ ]:
import os
from crewai import Agent
#from crewai_tools import ChatGroq  # Assuming ChatGroq is available
from langchain_groq import ChatGroq

# Set the environment variables (normally done outside code, e.g., in your system environment)
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'  # Model name for OpenAI (if needed)
os.environ["GROQ_API_KEY"] = get_groq_api_key()  # Function to securely retrieve your API key (from environment or config)

# Initialize Groq LLM using the environment variable for the model and API key
groq_model_name = os.environ.get('OPENAI_MODEL_NAME', 'gpt-4o-mini')  # Default to 'gpt-4o-mini' if not set
groq_api_key = os.environ.get("GROQ_API_KEY")  # Retrieves the API key


# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    #tools=[SerperDevTool(), ScrapeWebsiteTool()],
    tools=[search_tool, scrape_tool],
    #llm=groq_llm,
    #llm=hf_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    #tools=[SerperDevTool(), WebsiteSearchTool()],
    tools=[search_tool, scrape_tool],
    #llm=groq_llm,
    #llm=hf_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    #tools=[SerperDevTool(), WebsiteSearchTool()],
    tools=[search_tool, scrape_tool],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
 
    groq_llm = ChatGroq(
    groq_api_key=groq_api_key,  # Use your API key
    model_name="groq/llama-3.3-70b-versatile",  # The specific model to use
    temperature=0),
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: I need to gather the latest news and updates related to US inflation and its impact on the stock market for 2024. This will help me create a comprehensive report.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"US inflation news impact stock market 2024\"}"
## Tool Output: 
{'searchParameters': {'q': 'US inflation news

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
Inflation in the US cools down, leading to record
highs in the stock market! 📈 Explore the interplay
between inflation trends and market performance in
2024. Don't miss out on our latest insights!
--------------------------------------------------
LinkedIn
The evolving landscape of inflation rates and
stock market dynamics presents fresh opportunities
and challenges. Our latest analysis delves into
how investors can navigate changes in market
sentiment and make informed decisions in 2024.
Join the conversation on adapting to these
financial trends!
--------------------------------------------------
Facebook
2024 is shaping up to be a pivotal year for
investors as inflation rates drop, prompting
discussions around interest rate cuts and stock
market performance. Stay ahead with our detailed
insights and strategies for navigating these
economic changes!
--------------------------------------------------


C:\Users\Michela\AppData\Local\Temp\ipykernel_3984\3812343593.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  posts = result.pydantic.dict()['social_media_posts']


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

C:\Users\Michela\AppData\Local\Temp\ipykernel_3984\3784749904.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  display(Markdown(result.pydantic.dict()['article']))


# Impact of US Inflation on the Stock Market in 2024

## Overview of Current Economic Environment
The US economy in 2024 is navigating a complex landscape characterized by cooling inflation, fluctuating interest rates, and robust consumer sentiment. After reaching a peak of 7% in 2022, inflation rates have declined, with core PCE projected to settle at around 2.4% for 2024, down from 3.4% in 2023 (J.P. Morgan, 2024; Vanguard, 2024). This straitening of inflation has significant implications for equity markets, with the prospects of lower interest rates rekindling investment enthusiasm.

## Key Market Insights
- **Record Highs Following Inflation Drop**: As inflation rates continue to decline, stock markets have responded positively and reached record highs. Analysts anticipate that potential interest rate cuts from the Federal Reserve could make equities even more attractive, stimulating further investment (The Guardian, 2024).
- **Strong Consumer Spending**: Despite a rising unemployment rate, consumer spending remains resilient, particularly among affluent households, contributing to a steady economic climate (Morgan Stanley, 2024). This spending power is crucial in supporting various sectors, especially consumer discretionary and technology.
- **Volatility in Response to Economic Indicators**: Mixed signals from CPI reports have been a hallmark characteristic contributing to market volatility. As investor sentiment fluctuates based on expected Federal Reserve actions, volatility is expected to persist (AP News, 2024).
- **Policy Adjustments and Expectations**: The Federal Reserve’s continuous adjustments in monetary policy considerably influence stock market conditions. Investors must stay vigilant regarding Fed communications as these will guide investment decisions (US Bank, 2024).

## Implications for Stock Market Investments
- **Investment Strategies**: With anticipated interest rate cuts, investors should focus on sectors that typically benefit most from lower borrowing costs, particularly technology and consumer discretionary. These sectors thrive in a low-interest-rate environment, which bodes well for stock performance.
- **Defensive Positions**: Given ongoing economic uncertainty, a shift towards defensive positions in sectors such as utilities and consumer staples may help mitigate risks while providing consistent returns. These sectors generally hold up better during economic downturns.
- **Diversification**: Building a diversified portfolio that includes global equities, bonds, and alternative investments can serve as a buffer against potential market disruptions and enhance overall portfolio stability.

## Risks to Monitor
- **Inflation Volatility**: Unexpected spikes in inflation data could trigger abrupt market reactions. Investors must be adaptable, ready to adjust strategies based on real-time data as uncertainty looms (Deloitte, 2024).
- **Regulatory and Policy Changes**: Upcoming regulatory changes, particularly related to trade policy and economic initiatives influenced by the U.S. presidential election cycle, can have significant implications for market behavior and risk exposure (Charles Schwab, 2024).

## Recommendations for Content Creation
- **Thought Leadership Articles**: Focus on deep dives into how inflation trends can impact sectors specifically and provide actionable insights based on real-time data trends.
- **Infographics and Data Visualizations**: Create visual content to elucidate the relationship between inflation metrics, interest rates, and stock market performance, enhancing engagement with audiences.
- **Regular Market Updates**: Develop a consistent series of updates tracking inflation metrics throughout the year, supplemented by analysis indicating their short-term effects on market movements and investor sentiment.

## Conclusion
The relationship between inflation trends and stock market dynamics in 2024 presents opportunities for growth amid uncertainty. By closely following CPI reports and Federal Reserve announcements, investors can make informed decisions that secure their financial futures. Vigilance and strategic adaptation are key in navigating this evolving economic landscape.
